In [1]:
import pathlib
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import h5py
import pandas as pd 

from astropy.table import Table, QTable
from astropy.io import fits

%matplotlib inline

In [2]:
dat = Table.read('files/gaia_source_small.fits', format='fits')
gaia_source = dat.to_pandas()

In [3]:
# healpix level for each star (new column)
pd.options.mode.copy_on_write = True

level = 3
nside = hp.order2nside(level)
gaia_source["hp_pix_"+str(level)] = (gaia_source["source_id"] / (2**35 * 4**(12-level))).astype(int)

In [4]:
# --- cuts on primaries ---
# El-Badry+2021 quality cuts - only apply to primaries
primaries = gaia_source[(gaia_source['parallax']>1) & (gaia_source['parallax_error']<2) & \
                          (gaia_source['phot_g_mean_mag']!=np.nan)]                     
primaries = primaries[primaries['parallax_error']/primaries['parallax']<0.2] 

primaries['Gmag'] = primaries['phot_g_mean_mag']-5*np.log10(1/(primaries['parallax']*1e-3))+5

primaries = primaries[np.isfinite(primaries['phot_bp_mean_mag']) & \
                          np.isfinite(primaries['phot_rp_mean_mag']) & np.isfinite(primaries['Gmag'])]

# FGK dwarf cuts
primaries['g_rp'] = primaries['phot_g_mean_mag']-primaries['phot_rp_mean_mag']
primaries_FGK = primaries[(primaries['Gmag']>3.56) & (primaries['g_rp']>0.356) & \
                            (primaries['Gmag']<9.29) & (primaries['g_rp']<1.06)]

In [9]:
# --- cuts on primaries ---
# metallicity cuts - Andrae+ catalog
filename_path = 'files/table_1_catwise_andrae.fits'

# this function is Lucy's way to get rid of columns that are more than 1D (pandas doesn't support that)
def Table_to_pandas(fn):
    data = fits.open(fn)
    df = QTable(data[1].data)
    cols = []
    cols_drop = []
    for i in df.columns:
        if np.size(df[i][0])==1:
            cols.append(i)
        else:
            cols_drop.append(i)
    print(cols_drop)
    return df[cols].to_pandas()

df_andrae = Table_to_pandas(filename_path)

andrae_FGK_metal_poor = df_andrae[(df_andrae['mh_xgboost'] < -0.5) & (df_andrae['logg_xgboost'] > 3.7) & \
                                  (df_andrae['teff_xgboost'] < 6500) & (df_andrae['teff_xgboost'] > 4500)]

primaries_FGK_metal_poor = pd.merge(primaries_FGK,andrae_FGK_metal_poor,on='source_id')

[]


In [11]:
for i,row in primaries_FGK_metal_poor.iterrows():
    this_pix = row["hp_pix_"+str(level)] # iterrows turns everything into float64, so have to recast
    other_pix = hp.get_all_neighbours(nside, this_pix, nest=True) # get nside=8 nearest pixels

    mask = np.isin(
        gaia_source[f"hp_pix_{level}"], np.concatenate(([this_pix], other_pix))
    )
    subset = gaia_source[mask]

    print(subset)
    break

    # TODO: compute sky separation, El-Badry criteria (loosened), ∆G, ∆G-RP


                source_id         ra  ra_error        dec  dec_error  \
0              4295806720  44.996155  0.101618   0.005615   0.101334   
1             34361129088  45.004320  0.097320   0.021048   0.101753   
2             38655544960  45.004978  0.017885   0.019880   0.018772   
3            309238066432  44.995037  0.322039   0.038152   0.283505   
4            343597448960  44.963896  0.117176   0.043595   0.109004   
...                   ...        ...       ...        ...        ...   
726618  36028560796579072  44.983957  0.600125  19.436145   0.537773   
726619  36028595156145536  44.976937  0.079317  19.432122   0.064997   
726620  36028693940092160  45.018069  0.092331  19.450375   0.082299   
726621  36028693940092288  45.015733  0.271597  19.456914   0.240453   
726622  36028732595127168  44.990442  0.129504  19.443413   0.104056   

        parallax  parallax_error       pmra  pmra_error      pmdec  \
0       0.354331        0.122664  11.938352    0.137942  -4.08061

In [ ]:
df.to_parquet('df.parquet.gzip',

              compression='gzip') 

In [ ]:
with h5py.File(files[0], "r") as f:
    ra = f["ra"][:]
    dec = f["dec"][:]
    print(f.keys())

In [ ]:
plt.plot(ra, dec, "k.") # shape of first pixel? In first file? Is every file corresponding to a single pixel?

In [ ]:
check = gaia_source[0]
print(check)

print(check['ra'])

In [ ]:
import matplotlib.colors as mcolors

plt.hist2d(primaries['phot_bp_mean_mag']-primaries['phot_rp_mean_mag'],primaries['Gmag'],bins=100,\
           norm=mcolors.PowerNorm(0.3))
plt.gca().invert_yaxis()
plt.xlabel(r'BP-RP')
plt.ylabel(r'G mag')
plt.show()